In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, AveragePooling2D, Conv2D, Flatten
from keras.layers import Lambda
from keras import backend as K # tensorflow

import os
import coremltools
from coremltools.proto import NeuralNetwork_pb2

Using TensorFlow backend.


In [4]:
kears_file = "./KerasMNIST_quantization.h5"
coreml_file_template = './KerasMNIST_quantization_{}.mlmodel'

In [7]:
def build_and_learn_keras_model():


    batch_size = 128
    num_classes = 10
    epochs = 7

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print(x_train.shape)
    print(x_train.shape[1:])

    img_rows = 28
    img_cols = 28

    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # 入力値の正規化
    x_train /= 255
    x_test /= 255

    # 教師データをクラス分類のデータに変換
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # ネットワーク設計
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[6, 6], padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=[6, 6], padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # ネットワークの構成を出力する
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [8]:
if not os.path.exists(kears_file):
    keras_model = build_and_learn_keras_model()
    keras_model.save(kears_file)
else:
    keras_model = build_and_learn_keras_model()
    keras_model.save(kears_file)

(60000, 28, 28)
(28, 28)
(60000, 'train samples')
(10000, 'test samples')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        1184      
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        36896     
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
co

In [7]:
def quantize(model, name):
    print('--------------------------------------------------')
    print(name)
    print('--------------------------------------------------')
    quantizedModel = coremltools.models.neural_network.quantization_utils.quantize_weights(model, 8, name)
    # on macOS10.13 or earlier, a MLModel instance can not be returned.
    if type(quantizedModel) is coremltools.models.MLModel:
        print("ok")
        fileName = coreml_file_template.format(name)
        print(fileName)
        quantizedModel.save(fileName)
    else:
        print("error")

In [8]:
coreml_model = coremltools.converters.keras.convert(kears_file, input_names='image', output_names='digit')

coreml_model.author = u'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = u'Quantization'

coreml_model.input_description['image'] = u'入力画像'
coreml_model.output_description['digit'] = u'推定した数字の確率'

coreml_model.save(coreml_file_template.format('none'))

for name in ['linear', 'kmeans', 'linear_lut']:
    quantize(coreml_model, name)

0 : conv2d_5_input, <keras.engine.topology.InputLayer object at 0x131da5790>
1 : conv2d_5, <keras.layers.convolutional.Conv2D object at 0x131da5810>
2 : activation_5, <keras.layers.core.Activation object at 0x131da5bd0>
3 : conv2d_6, <keras.layers.convolutional.Conv2D object at 0x131da5890>
4 : activation_6, <keras.layers.core.Activation object at 0x12fad08d0>
5 : conv2d_7, <keras.layers.convolutional.Conv2D object at 0x12fabea90>
6 : activation_7, <keras.layers.core.Activation object at 0x12f95df50>
7 : conv2d_8, <keras.layers.convolutional.Conv2D object at 0x133025d10>
8 : conv2d_8__activation__, <keras.layers.core.Activation object at 0x13415de10>
9 : flatten_2, <keras.layers.core.Flatten object at 0x131b85d10>
10 : dense_3, <keras.layers.core.Dense object at 0x1324079d0>
11 : activation_8, <keras.layers.core.Activation object at 0x131b65c10>
12 : dense_4, <keras.layers.core.Dense object at 0x1324674d0>
13 : dense_4__activation__, <keras.layers.core.Activation object at 0x13415df10>